In [5]:
import monitoring
import logging
import time
import importlib
importlib.reload(monitoring)
from monitoring import Timer, ErrorCatcher

# A function to test timing; this one runs successfully.
@Timer()
def test_function():
    logging.info("Test function started")
    time.sleep(2)  # Simulate processing
    logging.info("Test function completed")
    return "Done!"

# A function to test error catching; it deliberately raises an exception.
@ErrorCatcher()
def test_exception():
    logging.info("Test exception function starting")
    # Intentionally raise an error to trigger the error catcher.
    raise ValueError("This is a test error!")

# Run test_function; its execution details will be logged in timing_results.csv and timing.log.
result = test_function()
print(result)

# Run test_exception in a try/except block to catch and print the error.
try:
    test_exception()
except Exception as e:
    print(f"Caught an exception as expected: {e}")

2025-03-20 08:08:12,442 - INFO - Test function started
2025-03-20 08:08:14,448 - INFO - Test function completed
2025-03-20 08:08:14,451 - INFO - Function `test_function` executed in 2.0084 sec, CPU Time: 0.0038 sec, Memory Change: 0.0000 MB, Final Memory: 131.1562 MB
2025-03-20 08:08:14,454 - INFO - Test exception function starting
2025-03-20 08:08:14,455 - ERROR - Function `test_exception` raised an exception: This is a test error!
Traceback (most recent call last):
  File "/Users/sra/files/projects/monitoring/monitoring.py", line 272, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/n3/q8wb235x57n_sx2q14dsngjr0000gn/T/ipykernel_98128/4159784902.py", line 21, in test_exception
    raise ValueError("This is a test error!")
ValueError: This is a test error!
2025-03-20 08:08:14,455 - ERROR - Function `test_exception` raised an exception: This is a test error!
Traceback (most recent call last):
  File "/Users/sra/files/projects/monitoring/m

Function `test_function` executed in 2.0084 sec, CPU Time: 0.0038 sec, Memory Change: 0.0000 MB, Final Memory: 131.1562 MB
Done!
Caught an exception as expected: This is a test error!


Test the parquet option:

In [8]:
import monitoring
import logging
import time
import importlib
importlib.reload(monitoring)
from monitoring import Timer, ErrorCatcher

# A function to test timing; this one runs successfully.
@Timer(results_format='parquet')
def test_function():
    logging.info("Test function started")
    time.sleep(2)  # Simulate processing
    logging.info("Test function completed")
    return "Done!"

# A function to test error catching; it deliberately raises an exception.
@ErrorCatcher(results_format='parquet')
def test_exception():
    logging.info("Test exception function starting")
    # Intentionally raise an error to trigger the error catcher.
    raise ValueError("This is a test error!")

# Run test_function; its execution details will be logged in timing_results.csv and timing.log.
result = test_function()
print(result)

# Run test_exception in a try/except block to catch and print the error.
try:
    test_exception()
except Exception as e:
    print(f"Caught an exception as expected: {e}")

2025-03-20 08:10:14,483 - INFO - Test function started
2025-03-20 08:10:16,489 - INFO - Test function completed
2025-03-20 08:10:16,491 - INFO - Function `test_function` executed in 2.0089 sec, CPU Time: 0.0037 sec, Memory Change: 0.2031 MB, Final Memory: 111.0469 MB
2025-03-20 08:10:16,505 - INFO - Test exception function starting
2025-03-20 08:10:16,506 - ERROR - Function `test_exception` raised an exception: This is a test error!
Traceback (most recent call last):
  File "/Users/sra/files/projects/monitoring/monitoring.py", line 272, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/n3/q8wb235x57n_sx2q14dsngjr0000gn/T/ipykernel_98128/2047492326.py", line 21, in test_exception
    raise ValueError("This is a test error!")
ValueError: This is a test error!
2025-03-20 08:10:16,506 - ERROR - Function `test_exception` raised an exception: This is a test error!
Traceback (most recent call last):
  File "/Users/sra/files/projects/monitoring/m

Function `test_function` executed in 2.0089 sec, CPU Time: 0.0037 sec, Memory Change: 0.2031 MB, Final Memory: 111.0469 MB
Done!
Caught an exception as expected: This is a test error!


In [9]:
import duckdb

timing_results = duckdb.sql("""
select * from read_parquet('logs/timing_results.parquet')
""").to_df()

timing_results.head()

,Timestamp,UUID,Function Name,Execution Time (s),CPU Time (sec),Memory Change (MB),Final Memory Usage (MB),Arguments,Log Message
0,2025-03-20 08:10:16,c7ebb846-3445-449b-9f1b-83d7064ba24b,test_function,2.008888,0.003663,0.203125,111.046875,"{""args"": [], ""kwargs"": {}}",Function `test_function` executed in 2.0089 se...
